In [8]:
import pandas as pd
import cv2
import os
import json
import string
import random as rand

In [9]:
CVAT_FOLDER_PATH = "/home/mh731nk/_data/experiments_tmp/data/revision_8/cvat_project_raw_unzip"

# Read video DataFrame 
Data was prepared in the previous part of the project

In [10]:
# Dataset path
df_videos = pd.read_pickle("/home/mh731nk/_data/experiments_tmp/data/revision_8/video.pkl", 'zip')

# Select videos for experiments

In [11]:
df_filtere_videos = df_videos.loc[df_videos["folder_class"].isin([
    'USG - Lung point',
    'USG - Lung sliding absent (aj ine znaky - radiol. klin.)',
    'USG - Lung sliding present (aj ine znaky - radiol. klin.)'
    ])]

print(f'LP -> {df_filtere_videos.loc[df_videos["folder_class"] == "USG - Lung point"].shape[0]}')
print(f'A -> {df_filtere_videos.loc[df_videos["folder_class"] == "USG - Lung sliding absent (aj ine znaky - radiol. klin.)"].shape[0]}')
print(f'P -> {df_filtere_videos.loc[df_videos["folder_class"] == "USG - Lung sliding present (aj ine znaky - radiol. klin.)"].shape[0]}')


LP -> 17
A -> 9
P -> 26


# Read anotations for videos

In [12]:
def id_generator(self, size: int = 8, chars: str = string.ascii_uppercase + string.digits) -> str:
    """
    Generate a random string identifier.

    This method generates a random string of the specified size using a secure random
    generator. By default, the generated string consists of uppercase letters and digits.

    Args:
        size (int): The length of the generated string. Defaults to 8.
        chars (str): A string containing the set of characters to choose from.
                     Defaults to uppercase letters and digits (A-Z, 0-9).

    Returns:
        str: A randomly generated string identifier.
    """
    return ''.join(random.SystemRandom().choice(chars) for _ in range(size))


Each video exported from the CVAT annotation tool is organized into a structured folder that holds both the raw video data and its associated metadata. This structure typically includes:

Raw Video Storage:
The primary video file is saved in a dedicated folder, ensuring that the original footage is preserved intact.

Metadata Files:
Accompanying the raw video are metadata files (often in JSON format) that provide detailed information about the video, such as resolution, frame rate, and other relevant properties.

Annotation Data:
In addition to video metadata, the export includes detailed annotation files. These files contain information about label masks and polygon coordinates for each frame, which outline the annotated regions of interest within the video. This structured annotation data is essential for tasks like object detection and segmentation.

This systematic folder organization makes it easy to access and process both the video content and its annotations, streamlining workflows for analysis and machine learning applications.

In [21]:
# Initialize a list to hold rows extracted from annotations
rows_to_append = []
print(f"Start processing of {df_filtere_videos.shape[0]} videos")
# Iterate over each row in the DataFrame containing filtered videos
for index, video_df_row in df_filtere_videos.iterrows():
    
    # Build the path to the 'data' folder within the current video's subfolder
    data_folder_path = os.path.join(
        CVAT_FOLDER_PATH, 
        video_df_row["video_subfolder_path"], 
        'data'
    )
    
    # Walk through the directory tree starting at data_folder_path.
    # os.walk returns tuples of (current_path, directories, files)
    folders_scan = [x for x in os.walk(data_folder_path)]
    
    # Retrieve the first subdirectory from the scan (assumes at least one exists)
    video_folder = folders_scan[0][1][0]
    
    # Print the name of the video folder (for debugging or logging purposes)
    # print(f' Video {video_df_row["video_id"]} is processing')
    
    # Construct the full path to the video subfolder
    path_video = os.path.join(
        CVAT_FOLDER_PATH,
        video_df_row["video_subfolder_path"]
    )
    
    # Open and load the annotations JSON file
    with open(os.path.join(path_video, 'annotations.json')) as json_file:
        json_anotation = json.load(json_file)
    
    # Open and load the task JSON file
    with open(os.path.join(path_video, 'task.json')) as json_file:
        task = json.load(json_file)
    
    # Iterate over each polygon shape in the first annotation object
    for polygon in json_anotation[0]['shapes']:
        # Create a dictionary with the extracted details for each polygon
        row = {
            'polygon_id': id_generator(8),               # Generate a unique identifier for the polygon
            'video_id': video_df_row["video_id"],        # Retrieve the video ID from the DataFrame row
            'name_cvat': task["name"],                   # Retrieve the task name from the task JSON
            'type': polygon["type"],                     # The type of shape (e.g., polygon, rectangle)
            'frame': int(polygon["frame"]),              # Convert the frame number to an integer
            'polygon_label': polygon["label"],           # The label assigned to the polygon (e.g., object class)
            'points': list(polygon["points"])            # List of coordinate points defining the polygon
        }
        # Append the dictionary to the list of rows to be processed later
        rows_to_append.append(row)

df_polygons = pd.DataFrame(rows_to_append) 
print(f'Dataset include {df_polygons.shape[0]} polygons.')

Start processing of 52 videos
Dataset include 2658 polygons.
